In [121]:
import pandas as pd
import datetime

#api
import praw

#pre-processing
import nltk 
from nltk.util import ngrams
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import MWETokenizer 
from nltk import word_tokenize
import re
# import string
#from textblob import TextBlob
#from nltk import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
from collections import Counter
# nltk.download()

from NLP_functions import display_topics

#analysis
from sklearn.decomposition import TruncatedSVD, NMF

ModuleNotFoundError: No module named 'NLP_functions'

In [3]:
reddit = praw.Reddit(client_id='oCxykEUclueAuA', client_secret='M8wVQw3nFxMsXHLGxurnTtkN-Mdvtg', user_agent='Crypto')

Version 7.0.0 of praw is outdated. Version 7.1.4 was released Sunday February 07, 2021.


#### Get top posts of all time

In [4]:
posts = []
ml_subreddit = reddit.subreddit('Cryptocurrency')

for post in list(ml_subreddit.top('all', limit=1000)):
    date = datetime.datetime.utcfromtimestamp(post.created_utc)
    posts.append([post.title, post.score, post.id, post.subreddit, post.url, post.num_comments, post.selftext, date])
    
posts = pd.DataFrame(posts,columns=['title', 'score', 'id', 'subreddit', 'url', 'num_comments', 'body', 'date'])
print(posts)

                                                 title  score      id  \
0    Elon Musk reminds me of a rich billionaire who...  39166  lce6zm   
1    CryptoNick is deleting all of his BitConnect v...  26507  7r0ftz   
2    Reminder: Robinhood blocked several stocks fro...  26220  lfrslo   
3    I will tell you exactly what is going on here,...  20143  7vga1y   
4    Robinhood is launching a Crypto Trading app to...  19969  7sx5ze   
..                                                 ...    ...     ...   
927                                 We Are All Satoshi   1487  gqt4ze   
928  Hackers Donate Stolen Bitcoin Worth $20,000 to...   1491  jeowcu   
929  Venezuelan money so worthless they just use it...   1485  bl34bp   
930  Your Guide to Monero, and Why It Has Great Pot...   1488  7ra409   
931             Top 101 Coins Grouped by Usage/Purpose   1484  lgeots   

          subreddit                                                url  \
0    CryptoCurrency  https://www.reddit.com/r/Cry

#### Get top coments from each top post

In [80]:
top_comments = []
for i,d in zip(posts.id, posts.date):
    try:
        submission = reddit.submission(id=i)
        submission.comments.replace_more(limit=0)
        comments=list([(comment.body) for comment in submission.comments.list()])
        comment_string = ''
        for x in comments[:20]:
            comment_string = comment_string + ' ' + x
        top_comments.append([comment_string,d])
    except:
        comments=None
top_comments = pd.DataFrame(top_comments,columns=['comment', 'date'])
print(top_comments)

                                               comment                date
0     If your life savings hinges on a Tweet from a... 2021-02-04 12:02:16
1     **Some Important Reminders:**\n\n* Even in th... 2018-01-17 11:46:08
2     It’s not even real Bitcoin.  You can never wi... 2021-02-09 01:55:15
3     [deleted] It's a shame i can vote this up onl... 2018-02-05 17:21:09
4     Reminder: all referral links are prohibited i... 2018-01-25 16:19:04
..                                                 ...                 ...
927   "I am the real Slim Shady" lol Except Craig W... 2020-05-26 08:28:25
928   Who is the true Robinhood here? > Bitcoin Wor... 2020-10-20 12:54:38
929   Lol, it's literally worth less than the paper... 2019-05-05 21:12:56
930   Great post ! You just forgot to mention https... 2018-01-18 14:42:02
931   Well written! Didnt know there was soo many s... 2021-02-09 22:02:05

[932 rows x 2 columns]


## Cleaning

combine comments by year

In [82]:
['year'] = top_comments['date'].dt.year
years = ['2017', '2018', '2019', '2020', '2021']
top_comments['year'].value_counts()

2018    405
2020    200
2021    123
2019    115
2017     89
Name: year, dtype: int64

In [106]:
comment_year = pd.DataFrame(index=years, columns=['Comments'])
for y in years:
    comment_string = ''
    for index, row in top_comments.iterrows():
        if str(row[2]) == y:
            comment_string = comment_string + ' ' + row[0]
            
    comment_string = re.sub(r'\([^)]*\)', '', comment_string) #remove links
    comment_string = re.sub('\w*\d\w*', '', comment_string) #remove numbers
    comment_string = re.sub(r'[^\w\s]', '', (comment_string))#remove punctuation
    comment_string = comment_string.lower() #lowercase
    comment_string = comment_string.replace('btc', 'bitcoin') #replace bitcoin acronym 
    
    #create stop words list
    stop = stopwords.words('english')
    stop += [ 'moderator', 'subreddit', 'reddit' , 'question', 'concern', 'please', 'contact', 'im', 
            'people', 'gon', 'seems', 'something', 'year', 'like', 'mod']
    stop = set(stop)

    mwe_tokenizer = MWETokenizer([ ('bull','run'), ('market','cap'), ('pump','dump'),
                             ('stock','market'), ('flaired', 'inaccurately'), ('get', 'rich', 'quick')])
    
    comment_token = mwe_tokenizer.tokenize(word_tokenize(comment_string))  # tokenize words

    #lemmatize words
    lemmatizer = nltk.stem.WordNetLemmatizer()
    comment_token_lemm = [(lemmatizer.lemmatize(w)) for w in comment_token]

    comment_token_lemm = [w for w in comment_token_lemm if w not in stop] #remove stop words
    
    comment_token_lemm = " ".join(comment_token_lemm)
    
    comment_year.loc[y, 'Comments'] = comment_token_lemm

In [107]:
print(comment_year)

                                               Comments
2017  longer version raccoon loses two piece candyfl...
2018  important reminder even case public figure str...
2019  twitter post random guy popular wa giving legi...
2020  offered cabinet position biden admin tldr form...
2021  life saving hinge tweet crazy billionaire dont...


In [ ]:
comment_year = re.sub(r'\([^)]*\)', '', all_comments) #remove links
comment_year = re.sub('\w*\d\w*', '', all_comments) #remove numbers
comment_year = re.sub(r'[^\w\s]', '', (all_comments))#remove punctuation
comment_year = comment_year.lower() #lowercase
comment_year = comment_year.replace('btc', 'bitcoin') #replace bitcoin acronym 

In [91]:
all_comments = comment_year.loc['2017', 'Comments'] #concatenate comments column into one string

all_comments = re.sub(r'\([^)]*\)', '', all_comments) #remove links
all_comments = re.sub('\w*\d\w*', '', all_comments) #remove numbers
all_comments = re.sub(r'[^\w\s]', '', (all_comments))#remove punctuation
all_comments = all_comments.lower() #lowercase
all_comments = all_comments.replace('btc', 'bitcoin') #replace bitcoin acronym 

#create stop words list
stop = stopwords.words('english')
stop += [ 'moderator', 'subreddit', 'reddit' , 'question', 'concern', 'please', 'contact', 'im', 
        'people', 'gon', 'seems', 'something', 'year', 'like', 'mod']
stop = set(stop)

mwe_tokenizer = MWETokenizer([ ('bull','run'), ('market','cap'), ('pump','dump'),
                             ('stock','market'), ('flaired', 'inaccurately'), ('get', 'rich', 'quick')])

words = mwe_tokenizer.tokenize(word_tokenize(all_comments))  # tokenize words

#lemmatize words
lemmatizer = nltk.stem.WordNetLemmatizer()
words = [(lemmatizer.lemmatize(w)) for w in words]

words = [w for w in words if w not in stop] #remove stop words

#create n grams
n = 2
bigrams = ngrams(words, n)

#count phrases
counter = Counter()
counter += Counter(bigrams)

for word, count in counter.most_common(30):
    print('%20s %i' % (" ".join(word), count))

           word word 28
           dont know 18
            ive seen 16
            let know 15
           month ago 15
     know compressed 14
   compressed thread 14
    thread commented 14
     commented today 14
           today faq 14
            faq full 14
           full list 14
         list source 14
         source code 14
        bitcoin cash 13
 acronym initialisms 12
initialisms abbreviation 12
abbreviation contraction 12
  contraction phrase 12
       phrase expand 12
       expand larger 12
          larger ive 12
         seen thread 12
        thread fewer 12
   fewer lettersmore 12
  lettersmore letter 12
 bitcoincoin bitcoin 12
         pretty much 12
            dont get 12
    ltccoin litecoin 11


In [ ]:
new = s.apply(fun) 

# Topic Modeling

### Count Vectorization

In [108]:
comments = list(comment_year['Comments'])

#comments_cleaned = [re.sub('\w*\d\w*', '', i) for i in comments]

# set(stopwords.words('english'))
cv = CountVectorizer(stop_words=stop)
X= cv.fit_transform(comments)
pd.DataFrame(X.toarray(), columns=cv.get_feature_names())

,__,___,____,______,_______,_________,__________,_______________________________,__coin,_banks_,...,zuckerberg,zuckerbucks,zuckercash,zunes,åtæreøm,în,ðoge,škoda,ебитес,ಠ_ಠ
0,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,1,2,0,0,1,1,1,1,0,0,...,1,0,2,0,0,1,0,1,1,1
2,0,0,0,0,0,0,0,0,0,1,...,1,1,0,1,1,0,0,0,0,0
3,0,0,0,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,2,0,0,0,0,0,0,0,...,0,0,0,0,0,0,4,0,0,0


#### Latent Semantic Analysis (LSA) with Count Vectorization

In [109]:
lsa = TruncatedSVD(3)
doc_topic = lsa.fit_transform(X)
lsa.explained_variance_ratio_

array([0.91871629, 0.03932098, 0.02240368])

In [110]:
topic_word = pd.DataFrame(lsa.components_.round(3),
             index = ["component_1","component_2", "component_3"],
             columns = cv.get_feature_names())
topic_word

,__,___,____,______,_______,_________,__________,_______________________________,__coin,_banks_,...,zuckerberg,zuckerbucks,zuckercash,zunes,åtæreøm,în,ðoge,škoda,ебитес,ಠ_ಠ
component_1,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.0,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
component_2,-0.001,-0.002,0.002,0.002,-0.001,-0.001,-0.001,-0.001,0.0,0.000,...,-0.000,0.000,-0.002,0.000,0.000,-0.001,0.003,-0.001,-0.001,-0.001
component_3,0.000,0.000,0.004,-0.002,0.000,0.000,0.000,0.000,-0.0,-0.001,...,-0.001,-0.001,0.000,-0.001,-0.001,0.000,0.008,0.000,0.000,0.000


In [111]:
def display_topics(model, feature_names, no_top_words, topic_names=None):
    for ix, topic in enumerate(model.components_):
        if not topic_names or not topic_names[ix]:
            print("\nTopic ", ix)
        else:
            print("\nTopic: '",topic_names[ix],"'")
        print(", ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))

In [112]:
display_topics(lsa, cv.get_feature_names(), 5)


Topic  0
crypto, bitcoin, wa, get, dont

Topic  1
bitcoin, buy, eth, doge, time

Topic  2
doge, coin, eth, buy, pump


In [113]:
Vt = pd.DataFrame(doc_topic.round(5),
             index = ['2017', '2018', '2019', '2020', '2021']s,
             columns = ["component_1","component_2", "component_3" ])
Vt

component_1  component_2  \
longer version raccoon loses two piece candyflo...    877.29575    101.97557   
important reminder even case public figure stri...   3992.80921   -304.81648   
twitter post random guy popular wa giving legit...   1039.75064    133.15514   
offered cabinet position biden admin tldr forme...   1588.36569    382.48774   
life saving hinge tweet crazy billionaire dont ...   1319.85405    289.14770   

                                                    component_3  
longer version raccoon loses two piece candyflo...     -8.97277  
important reminder even case public figure stri...      9.95274  
twitter post random guy popular wa giving legit...   -176.38796  
offered cabinet position biden admin tldr forme...   -166.91520  
life saving hinge tweet crazy billionaire dont ...    315.68207

#### Non-Negative Matrix Factorization (NMF)

In [114]:
nmf_model = NMF(3)
X_topic = nmf_model.fit_transform(X)

/Users/racheldilley/opt/anaconda3/lib/python3.8/site-packages/sklearn/decomposition/_nmf.py:1076: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn("Maximum number of iterations %d reached. Increase it to"


In [115]:
topic_word = pd.DataFrame(nmf_model.components_.round(3),
             index = ["component_1","component_2", "component_3"],
             columns = cv.get_feature_names())
topic_word

,__,___,____,______,_______,_________,__________,_______________________________,__coin,_banks_,...,zuckerberg,zuckerbucks,zuckercash,zunes,åtæreøm,în,ðoge,škoda,ебитес,ಠ_ಠ
component_1,0.016,0.031,0.000,0.000,0.016,0.016,0.016,0.016,0.001,0.001,...,0.016,0.001,0.031,0.001,0.001,0.016,0.000,0.016,0.016,0.016
component_2,0.000,0.000,0.000,0.024,0.000,0.000,0.000,0.000,0.003,0.007,...,0.006,0.007,0.000,0.007,0.007,0.000,0.000,0.000,0.000,0.000
component_3,0.000,0.000,0.031,0.000,0.000,0.000,0.000,0.000,0.002,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.062,0.000,0.000,0.000


In [116]:
display_topics(nmf_model, cv.get_feature_names(), 10)


Topic  0
crypto, wa, bitcoin, get, dont, money, coin, would, one, ha

Topic  1
bitcoin, wa, crypto, dont, money, time, get, one, would, buy

Topic  2
crypto, bitcoin, wa, get, doge, time, buy, dont, coin, money


In [117]:
H = pd.DataFrame(doc_topic.round(5),
             index = comments,
             columns = ["component_1","component_2", "component_3" ])
H

component_1  component_2  \
longer version raccoon loses two piece candyflo...      5.75540     15.20750   
important reminder even case public figure stri...     63.01190      0.00000   
twitter post random guy popular wa giving legit...      5.40992     33.66197   
offered cabinet position biden admin tldr forme...      0.93473     60.80170   
life saving hinge tweet crazy billionaire dont ...      1.72086      2.83304   

                                                    component_3  
longer version raccoon loses two piece candyflo...     10.31104  
important reminder even case public figure stri...      4.25597  
twitter post random guy popular wa giving legit...      0.00000  
offered cabinet position biden admin tldr forme...     13.11973  
life saving hinge tweet crazy billionaire dont ...     59.06013